| SMS Notifications | ISS (International Space Station) |
| ----------- | ----------- |
| Date | FEB 2023 |

In [9]:
# libs
import requests
import time
import pytz
import datetime

# config
import twilio_keys # comment out the other keys if you want this to work
# if you have two sets of keys in the config file this doesn't work for some reason
from twilio_keys import twilio_sid, twilio_auth_token

In [12]:
# replace the placeholder with your actual phone number
to_number = "+16199447143"

# replace the placeholder with your actual Twilio phone number
twilio_number = "+18556438915"

# latitude and longitude coordinates
my_location = (40.9807, -73.6837) # Rye, NY

def send_sms(body):
    response = requests.post(
        "https://api.twilio.com/2023-04-08/Accounts/{}/Messages.json".format(twilio_sid),
        auth=(twilio_sid, twilio_auth_token),
        data={
            "To": to_number,
            "From": twilio_number,
            "Body": body
        }
    )
    print(response.status_code)
    print(response.content)

def get_iss_passes():
    response = requests.get("http://api.open-notify.org/iss-pass.json",
                            params={"lat": my_location[0], "lon": my_location[1]})
    data = response.json()
    return data['response']

def notify_iss_passes():
    passes = get_iss_passes()
    current_time = datetime.datetime.now(pytz.utc)
    for pass_time in passes:
        rise_time = datetime.datetime.fromtimestamp(pass_time['risetime'], pytz.utc)
        if rise_time > current_time:
            time_to_rise = (rise_time - current_time).total_seconds()
            if time_to_rise <= 120:
                message = "The International Space Station will be flying over your location in 2 minutes!"
                send_sms(message)
                return

# if __name__ == "__main__":
#     while True:
#         notify_iss_passes()
#         time.sleep(30)